# Creating a minimal Blockchain
This notebook is based on [this blog post](https://medium.com/crypto-currently/lets-build-the-tiniest-blockchain-e70965a248b)

In [3]:
import hashlib as hasher
import datetime as date

In [4]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()

    def hash_block(self):
        sha = hasher.sha256()
        sha.update(str(self.index).encode('utf8') +
                   str(self.timestamp).encode('utf8') +
                   str(self.data).encode('utf8') +
                   str(self.previous_hash).encode('utf8'))
        return sha.hexdigest()


In [5]:
def create_genesis_block():
    '''
    Manually construct a block with
    index zero and arbitrary previous hash
    '''
    return Block(0, date.datetime.now(), "Genesis Block", "0")

In [6]:
def next_block(last_block):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_hash)

In [7]:
# Create the blockchain and add the genesis block
blockchain = [create_genesis_block()]
previous_block = blockchain[0]

# How many blocks should we add to the chain
# after the genesis block
num_of_blocks_to_add = 20

# Add blocks to the chain
for i in range(0, num_of_blocks_to_add):
    block_to_add = next_block(previous_block)
    blockchain.append(block_to_add)
    previous_block = block_to_add
    # Tell everyone about it!
    print ("Block #{} has been added to the blockchain!".format(block_to_add.index))
    print ("Hash: {}\n".format(block_to_add.hash) )

Block #1 has been added to the blockchain!
Hash: 67f435dc9b90c634d653af140ea546daa45a3007648f9788e4392eae9a4a84dd

Block #2 has been added to the blockchain!
Hash: 4a1d5073a934767439511e9136537ffab63cbcef90115691c76deb19d94197e9

Block #3 has been added to the blockchain!
Hash: 1855d15beb710bf955c5b3fc3373bca67d6ad306913318a07b76bfd8df0eeb06

Block #4 has been added to the blockchain!
Hash: 4fe99d631eeded5f1024a8f86539069178bf75d56e71be62bb41c177bf94f31c

Block #5 has been added to the blockchain!
Hash: a71c1dbe73bbf2063e233fb25b2e6a6af6f8fa48c53fd043898e15beeb3f7bc6

Block #6 has been added to the blockchain!
Hash: e7af81a1244ebcebec7e88883886adfc206f78d6c48ba8fdcd15a1879d4a3a87

Block #7 has been added to the blockchain!
Hash: 04aab91b4b78d5d208d390b9eb8690228c295c704fa7969534ff98368ade499e

Block #8 has been added to the blockchain!
Hash: 1776d232d5da5f71cfc124f893040bcacdb76c87e380ac4b8607a0d975be20df

Block #9 has been added to the blockchain!
Hash: 7833af4c25102fa61e853769313fda3

## Homework: 

Implement the rest of the system as on or more Python modules. The code is explained [here](https://medium.com/crypto-currently/lets-make-the-tiniest-blockchain-bigger-ac360a328f4d).

In [8]:
from flask import Flask
from flask import request
node = Flask(__name__)

this_node_transactions = []

@node.route('/txion', methods=['POST'])
def transaction():
    if request.method == 'POST':
        new_txion = request.get_json()
        this_node_transactions.append(new_txion)
        print ("New transaction")
        print("FROM: {}".format(new_txion['from']))
        print("TO: {}".format(new_txion['to']))
        print("AMOUNT: {}\n".format(new_txion['amount']))
        return "Transaction submission successful\n"

In [9]:
miner_address = "q3nf394hjg-random-miner-address-34nf3i4nflkn3oi"

def proof_of_work(last_proof):
    incrementor = last_proof + 1
    while not (incrementor % 9 == 0 and incrementor % last_proof == 0):
        incrementor += 1
    return incrementor

@node.route('/mine', methods = ['GET'])

def mine():
    last_block = blockchain[len(blockchain) - 1]
    last_proof = last_block.data['proof-of-work']
    proof = proof_of_work(last_proof)

    this_node_transactions.append(
        {"from": "network", "to": miner_address, "amount": 1}
    )
    new_block_data = {
        "proof-of-work": proof,
        "transactions": list(this_node_transactions)
    }

    new_block_index = last_block.index + 1
    new_block_timestamp = this_timestamp = date.datetime.now()
    last_block_hash = last_block.hash
    
    this_node_transactions[:] = []

    mined_block = Block(
        new_block_index,
        new_block_timestamp,
        new_block_data,
        last_block_hash
    )

    blockchain.append(mined_block)

    return json.dumps({
        "index": new_block_index,
        "timestamp": str(new_block_timestamp),
        "data": new_block_data,
        "hash": last_block_hash
    }) + "\n"

In [10]:
@node.route('/blocks', methods = ['GET'])
def get_blocks():
    chain_to_send = blockchain

    for block in chain_to_send:
        block_index = str(block.index)
        block_timestamp = str(block.timestamp)
        block_data = str(block.data)
        block_hash = block.hash
        block = {
            "index": block_index,
            "timestamp": block_timestamp,
            "data": block_data,
            "hash": block_hash
        }
    
    chain_to_send = json.dumps(chain_to_send)
    return chain_to_send

def find_new_chains():
    other_chains = []
    for node_url in peer_nodes:
        block = requests.get(node_url + "/blocks").content
        block = json.loads(block)
        other_chains.append(block)
    return other_chains

def consensus():
    other_chains = find_new_chains()
    longest_chain = blockchain
    for chain in other_chains:
        if len(longest_chain) < len(chain):
            longest_chain = chain
    blockchain = longest_chain